In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

In [ ]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = 'gemini-2.5-flash'

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very sarcastic, courteous chatbot. You try to agree with \
everything the other person says, but always with a little bit of sarcasm. If the other person is argumentative, \
you try to calm them down and keep chatting and more sarcastic."

gemini_system = "You are a very non-patient bot that, in order to get everyone in a good relationship, try to make\
the memebers of a conversation not to enter in conflict."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hey, guys?"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        conversation = f"""
        Alex: {gpt}
        Blake: {claude}
        Charlie: {gemini}
        """
        messages.append({"role": "user", "content": f"""
            You are Alex in a conversation with Blake and Charlie.
            The conversation so far is as follows:
            {conversation}
            Now with this, respond with what you would like to say next, as Alex.
            """
        })
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    #print(messages)
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        conversation = f"""
        Alex: {gpt}
        Blake: {claude_message}
        Charlie: {gemini}
        """
        messages.append({"role": "user", "content": f"""
            You are Blake in a conversation with Alex and Charlie.
            The conversation so far is as follows:
            {conversation}
            Now with this, respond with what you would like to say next, as Blake.
            """
        })
    # messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.
def call_gemini():
    user_prompt = []
    gemini = google.generativeai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=gemini_system
    )
    
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        conversation = f"""
            Alex: {gpt}
            Blake: {claude}
            Charlie: {gemini}
            """
    #print(conversation)    
    user_prompt.append(f"""
        You are Charlie in a conversation with Alex and Blake.
        The conversation so far is as follows:
        {conversation}
        Now with this, respond with what you would like to say next, as Charlie.
        """)
    #print(user_prompt)
    gemini = google.generativeai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=gemini_system
    )
    response = gemini.generate_content(user_prompt)
    return response.text


In [ ]:
call_gemini()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hey, guys?"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(3):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)